In [1]:
import cv2
import numpy as np
import mediapipe as mp
import argparse
import os
from datetime import datetime
import time
import math

# BVH creation utilities
class Joint:
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent
        self.children = []
        self.offset = np.zeros(3)
        self.channels = []
        self.motion = []
        
    def add_child(self, child):
        self.children.append(child)
        
class BVHSkeleton:
    def __init__(self):
        self.root = None
        self.joints = {}
        self.frames = 0
        self.frame_time = 1.0/30.0  # Default to 30fps
        
    def create_hierarchy(self):
        """Create a standard BVH hierarchy that matches MediaPipe landmarks"""
        
        # Create the root joint (Hips)
        self.root = Joint("Hips")
        self.joints["Hips"] = self.root
        
        # Create spine chain
        spine = Joint("Spine", self.root)
        self.root.add_child(spine)
        self.joints["Spine"] = spine
        
        chest = Joint("Chest", spine)
        spine.add_child(chest)
        self.joints["Chest"] = chest
        
        neck = Joint("Neck", chest)
        chest.add_child(neck)
        self.joints["Neck"] = neck
        
        head = Joint("Head", neck)
        neck.add_child(head)
        self.joints["Head"] = head
        
        # Create left arm
        l_collar = Joint("LeftCollar", chest)
        chest.add_child(l_collar)
        self.joints["LeftCollar"] = l_collar
        
        l_shoulder = Joint("LeftShoulder", l_collar)
        l_collar.add_child(l_shoulder)
        self.joints["LeftShoulder"] = l_shoulder
        
        l_elbow = Joint("LeftElbow", l_shoulder)
        l_shoulder.add_child(l_elbow)
        self.joints["LeftElbow"] = l_elbow
        
        l_wrist = Joint("LeftWrist", l_elbow)
        l_elbow.add_child(l_wrist)
        self.joints["LeftWrist"] = l_wrist
        
        # Create left hand (simplified)
        l_hand = Joint("LeftHand", l_wrist)
        l_wrist.add_child(l_hand)
        self.joints["LeftHand"] = l_hand
        
        # Create right arm
        r_collar = Joint("RightCollar", chest)
        chest.add_child(r_collar)
        self.joints["RightCollar"] = r_collar
        
        r_shoulder = Joint("RightShoulder", r_collar)
        r_collar.add_child(r_shoulder)
        self.joints["RightShoulder"] = r_shoulder
        
        r_elbow = Joint("RightElbow", r_shoulder)
        r_shoulder.add_child(r_elbow)
        self.joints["RightElbow"] = r_elbow
        
        r_wrist = Joint("RightWrist", r_elbow)
        r_elbow.add_child(r_wrist)
        self.joints["RightWrist"] = r_wrist
        
        # Create right hand (simplified)
        r_hand = Joint("RightHand", r_wrist)
        r_wrist.add_child(r_hand)
        self.joints["RightHand"] = r_hand
        
        # Create left leg
        l_hip = Joint("LeftHip", self.root)
        self.root.add_child(l_hip)
        self.joints["LeftHip"] = l_hip
        
        l_knee = Joint("LeftKnee", l_hip)
        l_hip.add_child(l_knee)
        self.joints["LeftKnee"] = l_knee
        
        l_ankle = Joint("LeftAnkle", l_knee)
        l_knee.add_child(l_ankle)
        self.joints["LeftAnkle"] = l_ankle
        
        l_foot = Joint("LeftFoot", l_ankle)
        l_ankle.add_child(l_foot)
        self.joints["LeftFoot"] = l_foot
        
        # Create right leg
        r_hip = Joint("RightHip", self.root)
        self.root.add_child(r_hip)
        self.joints["RightHip"] = r_hip
        
        r_knee = Joint("RightKnee", r_hip)
        r_hip.add_child(r_knee)
        self.joints["RightKnee"] = r_knee
        
        r_ankle = Joint("RightAnkle", r_knee)
        r_knee.add_child(r_ankle)
        self.joints["RightAnkle"] = r_ankle
        
        r_foot = Joint("RightFoot", r_ankle)
        r_ankle.add_child(r_foot)
        self.joints["RightFoot"] = r_foot
        
        # Setup channels for each joint
        self.setup_channels()
    
    def setup_channels(self):
        """Set up the channels for each joint in the hierarchy"""
        # Root has 6 channels: position and rotation
        self.root.channels = ["Xposition", "Yposition", "Zposition", "Zrotation", "Xrotation", "Yrotation"]
        
        # All other joints have 3 channels (rotation only)
        for name, joint in self.joints.items():
            if joint != self.root:
                joint.channels = ["Zrotation", "Xrotation", "Yrotation"]
    
    def update_joint_offsets(self, landmarks):
        """Calculate the initial offsets between joints based on the first frame of landmarks"""
        
        # Create a mapping from joint names to MediaPipe landmark indices
        # Based on MediaPipe's pose landmark model: https://google.github.io/mediapipe/solutions/pose.html
        mapping = {
            "Hips": 23,          # Left hip (will be centered)
            "Spine": 24,         # Right hip (will average with left hip)
            "Chest": 11,         # Left shoulder (midpoint with right)
            "Neck": 12,          # Right shoulder (midpoint for neck base)
            "Head": 0,           # Nose
            "LeftCollar": 11,    # Left shoulder
            "LeftShoulder": 11,  # Left shoulder
            "LeftElbow": 13,     # Left elbow
            "LeftWrist": 15,     # Left wrist
            "LeftHand": 19,      # Left hand (index finger MCP)
            "RightCollar": 12,   # Right shoulder
            "RightShoulder": 12, # Right shoulder
            "RightElbow": 14,    # Right elbow
            "RightWrist": 16,    # Right wrist
            "RightHand": 20,     # Right hand (index finger MCP)
            "LeftHip": 23,       # Left hip
            "LeftKnee": 25,      # Left knee
            "LeftAnkle": 27,     # Left ankle
            "LeftFoot": 31,      # Left foot index
            "RightHip": 24,      # Right hip
            "RightKnee": 26,     # Right knee
            "RightAnkle": 28,    # Right ankle
            "RightFoot": 32,     # Right foot index
        }
        
        # Scale to make the skeleton more appropriate size for BVH
        scale_factor = 100
        
        # First calculate the root position (centered between hips)
        left_hip = np.array([landmarks[23].x, landmarks[23].y, landmarks[23].z])
        right_hip = np.array([landmarks[24].x, landmarks[24].y, landmarks[24].z])
        hip_center = (left_hip + right_hip) / 2
        
        # Store original coordinates to calculate offsets
        coords = {}
        for name, idx in mapping.items():
            if idx is not None:
                coords[name] = np.array([landmarks[idx].x, landmarks[idx].y, landmarks[idx].z])
            
        # Calculate special midpoints
        coords["Spine"] = hip_center
        left_shoulder = coords["LeftShoulder"]  
        right_shoulder = coords["RightShoulder"]
        coords["Chest"] = (left_shoulder + right_shoulder) / 2
        coords["Neck"] = (left_shoulder + right_shoulder) / 2
        
        # Set the hip center at the origin
        for name, pos in coords.items():
            coords[name] = (pos - hip_center) * scale_factor
            # In BVH, Y is up, but in MediaPipe Y is down - flip Y
            coords[name][1] = -coords[name][1]
                
        # Calculate offsets based on joint hierarchy
        for name, joint in self.joints.items():
            if joint.parent is None:
                # Root joint
                joint.offset = np.zeros(3)
            else:
                # Child joints
                parent_name = joint.parent.name
                # For specific joints, use mediapipe's landmarks directly
                joint.offset = coords[name] - coords[parent_name]
    
    def quaternion_to_euler(self, q):
        """Convert quaternion to Euler angles (in degrees)"""
        # Extract quaternion components
        w, x, y, z = q
        
        # Roll (x-axis rotation)
        sinr_cosp = 2 * (w * x + y * z)
        cosr_cosp = 1 - 2 * (x * x + y * y)
        roll = math.atan2(sinr_cosp, cosr_cosp)
        
        # Pitch (y-axis rotation)
        sinp = 2 * (w * y - z * x)
        if abs(sinp) >= 1:
            pitch = math.copysign(math.pi / 2, sinp)  # Use 90 degrees if out of range
        else:
            pitch = math.asin(sinp)
            
        # Yaw (z-axis rotation)
        siny_cosp = 2 * (w * z + x * y)
        cosy_cosp = 1 - 2 * (y * y + z * z)
        yaw = math.atan2(siny_cosp, cosy_cosp)
        
        # Convert to degrees
        roll_deg = math.degrees(roll)
        pitch_deg = math.degrees(pitch)
        yaw_deg = math.degrees(yaw)
        
        return [yaw_deg, roll_deg, pitch_deg]  # ZXY order
    
    def get_bone_orientation(self, parent_pos, child_pos):
        """Calculate orientation from parent joint to child joint"""
        # Vector from parent to child
        direction = child_pos - parent_pos
        length = np.linalg.norm(direction)
        
        if length < 1e-6:  # Avoid division by zero
            return np.array([1, 0, 0, 0])  # Identity quaternion
        
        direction = direction / length
        
        # Original bone is assumed to point in the Y-axis in the rest pose
        original = np.array([0, 1, 0])
        
        # Quaternion to rotate from original to direction
        # First find rotation axis by cross product
        axis = np.cross(original, direction)
        axis_length = np.linalg.norm(axis)
        
        if axis_length < 1e-6:
            # If vectors are parallel (or anti-parallel)
            dot = np.dot(original, direction)
            if dot > 0.999:
                return np.array([1, 0, 0, 0])  # Identity quaternion
            else:
                # 180 degree rotation around X-axis
                return np.array([0, 1, 0, 0])
        
        # Normalize axis
        axis = axis / axis_length
        
        # Calculate rotation angle
        angle = math.acos(np.clip(np.dot(original, direction), -1.0, 1.0))
        
        # Create quaternion [w, x, y, z]
        s = math.sin(angle / 2)
        quat = np.array([
            math.cos(angle / 2),
            axis[0] * s,
            axis[1] * s,
            axis[2] * s
        ])
        
        return quat
    
    def process_frame(self, landmarks):
        """Process a frame of MediaPipe landmarks into joint rotations and positions"""
        frame_data = []
        
        # Define coordinates for each joint
        coords = {}
        
        # Get hip center (root position)
        left_hip = np.array([landmarks[23].x, landmarks[23].y, landmarks[23].z])
        right_hip = np.array([landmarks[24].x, landmarks[24].y, landmarks[24].z])
        hip_center = (left_hip + right_hip) / 2
        
        # Root position
        root_pos = np.array([
            hip_center[0] * 100, 
            -hip_center[1] * 100,  # Y is inverted in MediaPipe vs BVH
            hip_center[2] * 100
        ])
        
        # Add root position to frame data
        frame_data.extend(root_pos)
        
        # Define point locations for all joints
        # These are raw coordinates we'll use to calculate orientations
        points = {
            # Spine and head
            "Hips": hip_center,
            "Spine": (left_hip + right_hip) / 2,
            "LeftShoulder": np.array([landmarks[11].x, landmarks[11].y, landmarks[11].z]),
            "RightShoulder": np.array([landmarks[12].x, landmarks[12].y, landmarks[12].z]),
            "Chest": (np.array([landmarks[11].x, landmarks[11].y, landmarks[11].z]) + 
                      np.array([landmarks[12].x, landmarks[12].y, landmarks[12].z])) / 2,
            "Neck": (np.array([landmarks[11].x, landmarks[11].y, landmarks[11].z]) + 
                     np.array([landmarks[12].x, landmarks[12].y, landmarks[12].z])) / 2,
            "Head": np.array([landmarks[0].x, landmarks[0].y, landmarks[0].z]),
            
            # Left arm
            "LeftCollar": np.array([landmarks[11].x, landmarks[11].y, landmarks[11].z]),
            "LeftElbow": np.array([landmarks[13].x, landmarks[13].y, landmarks[13].z]),
            "LeftWrist": np.array([landmarks[15].x, landmarks[15].y, landmarks[15].z]),
            "LeftHand": np.array([landmarks[19].x, landmarks[19].y, landmarks[19].z]),
            
            # Right arm
            "RightCollar": np.array([landmarks[12].x, landmarks[12].y, landmarks[12].z]),
            "RightElbow": np.array([landmarks[14].x, landmarks[14].y, landmarks[14].z]),
            "RightWrist": np.array([landmarks[16].x, landmarks[16].y, landmarks[16].z]),
            "RightHand": np.array([landmarks[20].x, landmarks[20].y, landmarks[20].z]),
            
            # Left leg
            "LeftHip": np.array([landmarks[23].x, landmarks[23].y, landmarks[23].z]),
            "LeftKnee": np.array([landmarks[25].x, landmarks[25].y, landmarks[25].z]),
            "LeftAnkle": np.array([landmarks[27].x, landmarks[27].y, landmarks[27].z]),
            "LeftFoot": np.array([landmarks[31].x, landmarks[31].y, landmarks[31].z]),
            
            # Right leg
            "RightHip": np.array([landmarks[24].x, landmarks[24].y, landmarks[24].z]),
            "RightKnee": np.array([landmarks[26].x, landmarks[26].y, landmarks[26].z]),
            "RightAnkle": np.array([landmarks[28].x, landmarks[28].y, landmarks[28].z]),
            "RightFoot": np.array([landmarks[32].x, landmarks[32].y, landmarks[32].z]),
        }
        
        # Define the hierarchy and connectivity for rotation calculations
        hierarchy = {
            "Hips": ("Hips", "Spine"),
            "Spine": ("Spine", "Chest"),
            "Chest": ("Chest", "Neck"),
            "Neck": ("Neck", "Head"),
            "Head": ("Head", None),
            
            "LeftCollar": ("Chest", "LeftCollar"),
            "LeftShoulder": ("LeftCollar", "LeftElbow"),
            "LeftElbow": ("LeftElbow", "LeftWrist"),
            "LeftWrist": ("LeftWrist", "LeftHand"),
            "LeftHand": ("LeftHand", None),
            
            "RightCollar": ("Chest", "RightCollar"),
            "RightShoulder": ("RightCollar", "RightElbow"),
            "RightElbow": ("RightElbow", "RightWrist"),
            "RightWrist": ("RightWrist", "RightHand"),
            "RightHand": ("RightHand", None),
            
            "LeftHip": ("Hips", "LeftKnee"),
            "LeftKnee": ("LeftKnee", "LeftAnkle"),
            "LeftAnkle": ("LeftAnkle", "LeftFoot"),
            "LeftFoot": ("LeftFoot", None),
            
            "RightHip": ("Hips", "RightKnee"),
            "RightKnee": ("RightKnee", "RightAnkle"),
            "RightAnkle": ("RightAnkle", "RightFoot"),
            "RightFoot": ("RightFoot", None),
        }
        
        # Root rotation (global)
        # Calculate the orientation using shoulders and hips
        left_shoulder = points["LeftShoulder"]
        right_shoulder = points["RightShoulder"] 
        left_hip = points["LeftHip"]
        right_hip = points["RightHip"]
        
        # Calculate forward direction (perpendicular to the line between shoulders and hips)
        shoulder_center = (left_shoulder + right_shoulder) / 2
        hip_center = (left_hip + right_hip) / 2
        up_direction = shoulder_center - hip_center
        up_direction = up_direction / np.linalg.norm(up_direction)
        
        right_direction = right_shoulder - left_shoulder
        right_direction = right_direction / np.linalg.norm(right_direction)
        
        forward_direction = np.cross(right_direction, up_direction)
        forward_direction = forward_direction / np.linalg.norm(forward_direction)
        
        # Recalculate right to ensure orthogonal basis
        right_direction = np.cross(up_direction, forward_direction)
        
        # Create rotation matrix for root
        rot_matrix = np.vstack([right_direction, up_direction, forward_direction]).T
        
        # Extract Euler angles
        # Simple approach: assumes the rotation matrix is orthogonal
        yaw = math.atan2(rot_matrix[2, 0], rot_matrix[0, 0])
        pitch = math.atan2(-rot_matrix[1, 2], rot_matrix[1, 1])
        roll = math.atan2(-rot_matrix[2, 1], rot_matrix[2, 2])
        
        # Convert to degrees
        root_rotation = [
            math.degrees(yaw),     # Z
            math.degrees(roll),    # X
            math.degrees(pitch)    # Y
        ]
        
        # Add root rotation
        frame_data.extend(root_rotation)
        
        # Calculate orientations for all other joints
        for name, joint in self.joints.items():
            if joint == self.root:
                # Already handled root above
                continue
            
            # Get the points to calculate orientation from
            source_name, target_name = hierarchy[name]
            
            if target_name is None:
                # End sites just use parent rotation
                source_name = joint.parent.name
                target_name = name
                # Default rotation for end sites
                rotation = [0, 0, 0]
            else:
                # Get parent and child positions
                parent_pos = points[source_name]
                child_pos = points[target_name]
                
                # Get quaternion orientation
                quat = self.get_bone_orientation(parent_pos, child_pos)
                
                # Convert to Euler angles
                rotation = self.quaternion_to_euler(quat)
            
            # Add joint rotation to frame data
            frame_data.extend(rotation)
        
        return frame_data
    
    def write_bvh(self, filename):
        """Write the skeleton and motion data to a BVH file"""
        with open(filename, 'w') as f:
            # Write header
            f.write("HIERARCHY\n")
            
            # Write the skeleton hierarchy recursively
            self._write_joint(f, self.root, 0)
            
            # Write motion data
            f.write("MOTION\n")
            f.write(f"Frames: {self.frames}\n")
            f.write(f"Frame Time: {self.frame_time}\n")
            
            # Calculate total number of channels
            total_channels = sum(len(joint.channels) for joint in self.joints.values())
            
            # Prepare motion data
            motion_data = []
            for frame in range(self.frames):
                frame_values = []
                
                for name, joint in self.joints.items():
                    if frame < len(joint.motion):
                        # Add motion data for this joint
                        frame_values.extend(joint.motion[frame])
                    else:
                        # Add zeros if missing data
                        frame_values.extend([0.0] * len(joint.channels))
                
                # Make sure we have the right number of values
                if len(frame_values) < total_channels:
                    frame_values.extend([0.0] * (total_channels - len(frame_values)))
                elif len(frame_values) > total_channels:
                    frame_values = frame_values[:total_channels]
                    
                motion_data.append(frame_values)
            
            # Write frame data
            for frame_values in motion_data:
                f.write(" ".join(map(lambda x: f"{x:.6f}", frame_values)) + "\n")
    
    def _write_joint(self, f, joint, indent_level):
        """Write a single joint to the BVH file"""
        indent = "  " * indent_level
        
        if joint.parent is None:
            # Root joint
            f.write(f"{indent}ROOT {joint.name}\n")
        else:
            f.write(f"{indent}JOINT {joint.name}\n")
        
        f.write(f"{indent}{{\n")
        
        # Write offset
        offset_str = " ".join(map(lambda x: f"{x:.6f}", joint.offset))
        f.write(f"{indent}  OFFSET {offset_str}\n")
        
        # Write channels
        channels_str = " ".join(joint.channels)
        f.write(f"{indent}  CHANNELS {len(joint.channels)} {channels_str}\n")
        
        # Write children
        for child in joint.children:
            self._write_joint(f, child, indent_level + 1)
        
        # If this is an end site with no children, write an End Site
        if not joint.children:
            f.write(f"{indent}  End Site\n")
            f.write(f"{indent}  {{\n")
            # Typically end sites have a small positive Y offset
            f.write(f"{indent}    OFFSET 0.00 5.00 0.00\n")
            f.write(f"{indent}  }}\n")
        
        f.write(f"{indent}}}\n")


def process_video(input_file, output_file, visualize=False, debug=False, downsample_factor=1):
    """
    Process a video file to extract pose data and convert to BVH
    
    Args:
        input_file: Path to input video file
        output_file: Path to output BVH file
        visualize: Whether to show visualization
        debug: Whether to print debug information
        downsample_factor: Factor to downsample frames (1 = use all frames)
    """
    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    
    # Create BVH skeleton
    skeleton = BVHSkeleton()
    
    # Verify file exists
    if not os.path.exists(input_file):
        print(f"Error: Input file '{input_file}' does not exist.")
        return False
    
    # Open video file
    print(f"Opening video file: {input_file}")
    cap = cv2.VideoCapture(input_file)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video file '{input_file}'.")
        print("Supported formats include: mp4, avi, mov, etc.")
        return False
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 30  # Default if unable to determine
        print(f"Warning: Could not determine video FPS, using default of {fps}")
    
    # Set frame time accounting for downsampling
    effective_fps = fps / downsample_factor
    skeleton.frame_time = 1.0 / effective_fps
    
    # Count frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Video info: {width}x{height}, {fps} FPS, {total_frames} frames")
    if downsample_factor > 1:
        print(f"Downsampling: Using 1 frame per {downsample_factor} frames (effective FPS: {effective_fps:.2f})")
    
    if total_frames <= 0:
        print("Warning: Could not determine total frame count")
        total_frames = float('inf')  # Process until end of video
    
    # Initialize variables
    frame_count = 0
    processed_count = 0
    is_first_frame_with_pose = True
    start_time = time.time()
    
    # Create the skeleton hierarchy
    skeleton.create_hierarchy()
    
    # Setup Pose detection
    print("Initializing MediaPipe pose detection...")
    
    # Resolution affects performance, so choose model complexity wisely
    if width * height <= 640 * 480:
        model_complexity = 2  # Best accuracy for small videos
    elif width * height <= 1280 * 720:
        model_complexity = 1  # Balance for HD
    else:
        model_complexity = 1  # Performance for large videos
    
    pose_config = {
        'static_image_mode': False,
        'model_complexity': model_complexity,
        'min_detection_confidence': 0.5,
        'min_tracking_confidence': 0.5,
        'smooth_landmarks': True
    }
    
    if debug:
        print(f"MediaPipe pose config: {pose_config}")
    
    # Create output directory if it doesn't exist
    output_dir = os.path.dirname(output_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Process the video
    with mp_pose.Pose(**pose_config) as pose:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break
            
            frame_count += 1
            
            # Skip frames if downsampling
            if downsample_factor > 1 and frame_count % downsample_factor != 0:
                continue
            
            if debug or frame_count % 100 == 0:
                print(f"Processing frame {frame_count}/{total_frames}")
            
            # Convert the BGR image to RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # For better performance, mark the image as not writeable
            image_rgb.flags.writeable = False
            
            # Process the image and detect pose
            try:
                results = pose.process(image_rgb)
            except Exception as e:
                print(f"Error processing frame {frame_count}: {e}")
                continue
            
            # Check if pose landmarks are detected
            if results.pose_landmarks:
                processed_count += 1
                
                # If this is the first frame with good landmarks, use it to initialize joint offsets
                if is_first_frame_with_pose:
                    print("First good pose detected! Initializing joint offsets...")
                    skeleton.update_joint_offsets(results.pose_landmarks.landmark)
                    is_first_frame_with_pose = False
                
                # Process frame
                try:
                    motion_data = skeleton.process_frame(results.pose_landmarks.landmark)
                except Exception as e:
                    print(f"Error processing motion data for frame {frame_count}: {e}")
                    print(f"Exception details: {str(e)}")
                    continue
                
                # Store frame data
                channel_index = 0
                for joint_name, joint in skeleton.joints.items():
                    if not hasattr(joint, 'motion'):
                        joint.motion = []
                    
                    # Extract just this joint's channels
                    channel_count = len(joint.channels)
                    joint_data = motion_data[channel_index:channel_index + channel_count]
                    channel_index += channel_count
                    
                    joint.motion.append(joint_data)
                
                # Draw the pose landmarks for visualization
                if visualize:
                    image_rgb.flags.writeable = True
                    
                    # Draw pose landmarks
                    mp_drawing.draw_landmarks(
                        image,
                        results.pose_landmarks,
                        mp_pose.POSE_CONNECTIONS,
                        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                    )
                    
                    # Add frame info
                    cv2.putText(
                        image, f"Frame: {frame_count}/{total_frames}", 
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2
                    )
                    
                    cv2.putText(
                        image, f"Processed: {processed_count} frames", 
                        (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2
                    )
                    
                    # Display the image


    # Release resources
    cap.release()
    if visualize:
        cv2.destroyAllWindows()
    
    # Duration and statistics
    duration = time.time() - start_time
    
    print(f"\nProcessing complete: {processed_count} frames with pose data out of {frame_count} total frames")
    print(f"Time taken: {duration:.2f} seconds ({frame_count/duration:.2f} FPS)")
    
    # Check if we have any processed frames
    if processed_count == 0:
        print("\nERROR: No frames were successfully processed!")
        print("Possible issues:")
        print("1. The video doesn't contain a clearly visible person")
        print("2. The lighting conditions make pose detection difficult")
        print("3. The person is too small in the frame or too far from the camera")
        print("4. The video format might be incompatible")
        print("\nTry with a different video or with the person closer to camera")
        return False
    
    # Set the total number of frames in the BVH file
    skeleton.frames = processed_count
    
    # Write BVH file
    try:
        print(f"Writing BVH file to {output_file}...")
        skeleton.write_bvh(output_file)
        print(f"BVH file written successfully with {processed_count} frames of motion data")
        return True
    except Exception as e:
        print(f"Error writing BVH file: {e}")
        return False


2025-02-26 16:50:14.033580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740617414.050406   26509 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740617414.055815   26509 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 16:50:14.072904: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
filename = "fight2"
process_video(f"{filename}.mp4", f"{filename}.bvh", True, False)

Opening video file: fight2.mp4
Video info: 1280x720, 30.0 FPS, 43 frames
Initializing MediaPipe pose detection...
First good pose detected! Initializing joint offsets...


I0000 00:00:1740617417.383400   26509 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1740617417.385594   26604 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1740617417.451052   26582 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740617417.483189   26584 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740617417.502588   26582 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.



Processing complete: 43 frames with pose data out of 43 total frames
Time taken: 1.23 seconds (35.07 FPS)
Writing BVH file to fight2.bvh...
BVH file written successfully with 43 frames of motion data


True